# Behaviour

Modules from low frequency to high frequency:

4-01

(Modules in path planning are boxed)

* Behaviour planner has to incorporate much data (timescales 10s+)

#### Sample output:

```
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": null,
    "target_speed" : 15.0,
    "seconds_to_reach_target" : 10.0,
}
```

### The Behaviour Problem

4-02

Behaviour planner: 
- Takes map, route and predictions as input
- Output: suggest states (maneuvers) that are 

Trajectory planner: 
- Responsible for executing maneuvers in a collision-free, smooth and safe way
Driver executes and is responsible for safety (if navigator issues instructions that will immediately result in a crash, the driver will wait to execute)
Navigator 


### A solution: Finite State Machines

4-03

- Discrete states connected by transitions
- Begin at some start state
- Decide to go to which state to transition to using a state transition function

Strengths:
- Self-documenting, easy to reason about: map logical state to physical state
- Maintainable (for small state spaces)

Weaknesses:
- Easily abused: may just continue adding new states if problem changes or you discover things you hadn't considered before, leading to sloppy code
- Not maintainable (for large state spaces)


### States for self-driving cars


- Tradeoff between wanting a small, maintainable state space and wanting to include all the states we need

Sample list of state spaces for highway driving:

(Ideation process: Brainstorm list of ideas and prune list)

Pruned to 
- Keep lane
- Change lane left
- Change lane right
- Prepare lane change left
    - Needed because pre-change lane left,
        - it's safer to match the speed of the left lane
        - without this manouver, you'd have to just wait for a gap to appear in the left lane
        - Unclear when to turn on the turn signal (ideally want to turn it on a few seconds before lane change)
- Prepare lane change right

(Pruned many states because they are implementations of e.g. keep lane, change lane left/right).

#### State details (in Frenet coordinates)

4-05

### Transition Functions

#### Inputs
* Predictions, map, speed limit, localisation data, current state

#### Implementation (Pseudocode)
- Generate rough trajectories for each accessible 'next state' and select the state with the trajectory with the lowest cost.

In [1]:
def transition_function(predictions, current_fsm_state, current_pose, cost_functions, weights):
    # only consider states which can be reached from current FSM state.
    possible_successor_states = successor_states(current_fsm_state)

    # keep track of the total cost of each state.
    costs = []
    for state in possible_successor_states:
        # generate a rough idea of what trajectory we would
        # follow IF we chose this state.
        trajectory_for_state = generate_trajectory(state, current_pose, predictions)

        # calculate the "cost" associated with that trajectory.
        cost_for_state = 0
        for i in range(len(cost_functions)) :
            # apply each cost function to the generated trajectory
            cost_function = cost_functions[i]
            cost_for_cost_function = cost_function(trajectory_for_state, predictions)

            # multiply the cost by the associated weight
            weight = weights[i]
            cost_for_state += weight * cost_for_cost_function
        costs.append({'state' : state, 'cost' : cost_for_state})

    # Find the minimum cost state.
    best_next_state = None
    min_cost = 9999999
    for i in range(len(possible_successor_states)):
        state = possible_successor_states[i]
        cost  = costs[i]
        if cost < min_cost:
            min_cost = cost
            best_next_state = state 

    return best_next_state

### Designing cost functions

#### 1. Vehicle speed
* Want te get to destination quickly but don't want to break the law.
    * Suppose maximum cost if you're above the speed limit
    * Zero cost at
    * Cost of not moving: bad but not as bad as exceeding the speed limit.
    * Arbitrarily connect points we've plotted with a linear function
    * May want to parameterise these
    
4-06

#### 2. Lane Choice

Options:
* Lane Change (LC)
* Keep Lane (KL)

4-07

Establish relevant variables:
* $\Delta s = s_G - s$
    * Longitudinal distance the vehicle has before it needs to get to the goal lane
    * Cost should be inversely propertional to $\Delta s$ (lane change costs more important when we're closer to the goal because it's more likely we won't make it in time)
* $\Delta d = d_G - d_{LC/KL}$
    * lateral distance between goal lane and options being considered
    * Cost should be propertional to $\Delta d$ 

* Want to normalise cost such that cost is always in interval [0,1].
    * choose e.g. $ \text{cost} = 1 - e^{-\frac{|\Delta d|}{\Delta s}}$
    
    
#### Discussion of cost function design

4-08

* New problems like not being aggressive enough about turning left at traffic lights
* Regression testing: define some set of test cases (situations with corresponding expected behaviour) and testing them when redesigning cost functions
* Ideally each cost function has a specific function
    * -> e.g. Define a few cost functinos associated with speed (one for safety, one for legality) 
    
4-09

* Different priorities in different situations
    * add 'obeys traffic rules' cost function if at an intersection and traffic light just turned yellow.
    
### Scheduling Compute Time
* Behaviour module decisions take longer time and don't change as quickly.

* Behaviour needs info from prediction and localisation to begin its second cycle
    * But prediction in the middle of an update cycle.
    * Use slightly-out-of-date data so we don't block the pipeline for downsteram components (which have higher update frequency)
    
### Implement a Behaviour planner
